In [2]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib as plt

In [7]:

# Load the two datasets
df1 = pd.read_csv("./data/student_dataset.csv")
df2 = pd.read_csv("./data/scholarship_dataset.csv")

# Concatenate the datasets horizontally
combined_df = pd.concat([df1, df2], axis=1)

# Shuffle the rows of the combined dataset
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the shuffled and combined DataFrame to a new CSV file
combined_df.to_csv("combined_dataset.csv", index=False)